#### Comprobar los puertos

In [ ]:
import socket

def check_port(host, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(5)
    try:
        s.connect((host, port))
        return f'Conexion establecida correctamente'
    except socket.error as e:
        return f'Conexion fallida'
    finally:
        s.close()

print(check_port("kafka-azure.norwayeast.cloudapp.azure.com", 29090))
print(check_port("kafka-azure.norwayeast.cloudapp.azure.com", 9091))
print(check_port("kafka-azure.norwayeast.cloudapp.azure.com", 9090))


Connection to kafka-azure.norwayeast.cloudapp.azure.com:29090 failed: timed out
False
True
True


#### Installs

In [ ]:
%pip install kafka
%pip install fastavro
%pip install requests
%pip install influxdb-client

dbutils.library.restartPython()

In [ ]:
%pip install --upgrade kafka-python
dbutils.library.restartPython()

#### Imports

In [ ]:
import os
import threading
import fastavro
import json
import influxdb_client
import datetime as dt
from kafka import KafkaConsumer
from io import BytesIO
from fastavro.schema import parse_schema
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import requests
import time


### Version un solo consumidor

In [ ]:

#::::::::::::::::ESQUEMA::::::::::::::
schema_url = requests.get('{}/subjects/{}/versions/latest/schema'.format("http://kafka-azure.norwayeast.cloudapp.azure.com:8081", "criptomonedas"))
avro_schema = schema_url.json()
parsed_schema = parse_schema(avro_schema)

influxURL = "https://us-east-1-1.aws.cloud2.influxdata.com/"
influxToken = "TmlX0yIaYqlwgv5zwkVyMfGaTzKTJP1AvylAuEahfytsgTraUygURyXcblk4jE_GGCOIgUTKJzKT2fk48XzJFA=="
organization = "Viewnext"

print(parsed_schema)

#::::CONFIGURACION DE LOS CONSUMIDORES:::
conf = {
  "bootstrap_servers": ['kafka-azure.norwayeast.cloudapp.azure.com:9092', 'kafka-azure.norwayeast.cloudapp.azure.com:9091', 'kafka-azure.norwayeast.cloudapp.azure.com:9090'],
  "auto_offset_reset": 'earliest',
  "enable_auto_commit": True,
  "group_id": 'Monedas'
}

consumidor_1 = KafkaConsumer(
       "criptomonedas",
       client_id="Databricks_1",
       **conf,
)

#::::CONFIGURACION DE LOS INFLUXDB:::

influxdb_client = InfluxDBClient(
  url=influxURL, 
  token=influxToken, 
  org=organization
)

bucket="criptobucket"
write_api = influxdb_client.write_api(write_options=SYNCHRONOUS)

# Función para deserializar mensajes Avro
def avro_decoder(message_value):
    bytes_reader = BytesIO(message_value)
    reader = fastavro.reader(bytes_reader, parsed_schema)
    for mensaje in reader:
        return mensaje

# Función para consumir mensajes y escribir en InfluxDB
def ver_mensajes(consumer):
  for message in consumer:
    mensaje = avro_decoder(message.value)
    print(f"El {consumer.config['client_id']} se ha ocupado del: {mensaje}\n")

    tiempo_actual = dt.datetime.utcnow()
    timestamp = tiempo_actual.isoformat() + 'Z'  # Añadir 'Z' para indicar que es en formato UTC
    registro = (
      Point("criptomonedas")
      .tag("Coin", mensaje['nombre'])
      .field("Bid", mensaje['bid'])
      .field("Bid_size", mensaje['bid_size'])
      .field("Ask", mensaje['precio_ultimo'])
      .field("Ask_size", mensaje['precio_ultimo'])
      .field("Daily_change", mensaje['daily_change'])
      .field("Volume", mensaje['volume'])
      .field("Price_max", mensaje['precio_maximo'])
      .field("Price_min", mensaje['precio_minimo'])
      .field("Price", mensaje['precio_ultimo'])).time(timestamp)
    
    write_api.write(bucket=bucket, org=organization, record=registro)

while True:
    ver_mensajes(consumidor_1)
    time.sleep(5)


{'type': 'record', 'name': 'CryptoCurrency', 'fields': [{'name': 'nombre', 'type': 'string'}, {'name': 'bid', 'type': 'double'}, {'name': 'bid_size', 'type': 'double'}, {'name': 'ask', 'type': 'double'}, {'name': 'ask_size', 'type': 'double'}, {'name': 'daily_change', 'type': 'double'}, {'name': 'daily_change_percentage', 'type': 'double'}, {'name': 'precio_ultimo', 'type': 'double'}, {'name': 'volume', 'type': 'double'}, {'name': 'precio_maximo', 'type': 'double'}, {'name': 'precio_minimo', 'type': 'double'}], '__fastavro_parsed': True, '__named_schemas': {'CryptoCurrency': {'type': 'record', 'name': 'CryptoCurrency', 'fields': [{'name': 'nombre', 'type': 'string'}, {'name': 'bid', 'type': 'double'}, {'name': 'bid_size', 'type': 'double'}, {'name': 'ask', 'type': 'double'}, {'name': 'ask_size', 'type': 'double'}, {'name': 'daily_change', 'type': 'double'}, {'name': 'daily_change_percentage', 'type': 'double'}, {'name': 'precio_ultimo', 'type': 'double'}, {'name': 'volume', 'type': 'do

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:714)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:430)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:430)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

### Version 2 o N consumidores (hilos)

In [ ]:

#::::::::::::::::ESQUEMA::::::::::::::
schema_url = requests.get('{}/subjects/{}/versions/latest/schema'.format("http://kafka-azure.norwayeast.cloudapp.azure.com:8081", "criptomonedas"))
avro_schema = schema_url.json()
parsed_schema = parse_schema(avro_schema)

influxURL = "https://us-east-1-1.aws.cloud2.influxdata.com/"
influxToken = "TmlX0yIaYqlwgv5zwkVyMfGaTzKTJP1AvylAuEahfytsgTraUygURyXcblk4jE_GGCOIgUTKJzKT2fk48XzJFA=="
organization = "Viewnext"

print(parsed_schema)

#::::CONFIGURACION DE LOS CONSUMIDORES:::
conf = {
  "bootstrap_servers": ['kafka-azure.norwayeast.cloudapp.azure.com:9092', 'kafka-azure.norwayeast.cloudapp.azure.com:9091', 'kafka-azure.norwayeast.cloudapp.azure.com:9090'],
  "auto_offset_reset": 'earliest',
  "enable_auto_commit": True,
  "group_id": 'Coins'
}

consumidor_1 = KafkaConsumer(
       "criptomonedas",
       client_id="Databricks_1",
       **conf,
)

consumidor_2 = KafkaConsumer(
       "criptomonedas",
       client_id="Databricks_2",
       **conf,
)


influxdb_client = InfluxDBClient(
  url=influxURL, 
  token=influxToken, 
  org=organization
)

bucket="criptobucket"
write_api = influxdb_client.write_api(write_options=SYNCHRONOUS)

# Función para deserializar mensajes Avro
def avro_decoder(message_value):
    bytes_reader = BytesIO(message_value)
    reader = fastavro.reader(bytes_reader, parsed_schema)
    for mensaje in reader:
        return mensaje

# Función para consumir mensajes y escribir en InfluxDB
def ver_mensajes(consumer):
  for message in consumer:
    mensaje = avro_decoder(message.value)
    print(f"El {consumer.config['client_id']} se ha ocupado del: {mensaje}\n")
    
    #tiempo_actual = dt.datetime.now()
    #timestamp = int(tiempo_actual.timestamp())
    tiempo_actual = dt.datetime.utcnow()
    timestamp = tiempo_actual.isoformat() + 'Z'  # Añadir 'Z' para indicar que es en formato UTC
    registro = (
      Point("criptomonedas2")
      .tag("Coin", mensaje['nombre'])
      .field("Bid", mensaje['bid'])
      .field("Bid_size", mensaje['bid_size'])
      .field("Ask", mensaje['precio_ultimo'])
      .field("Ask_size", mensaje['precio_ultimo'])
      .field("Daily_change", mensaje['daily_change'])
      .field("Volume", mensaje['volume'])
      .field("Price_max", mensaje['precio_maximo'])
      .field("Price_min", mensaje['precio_minimo'])
      .field("Price", mensaje['precio_ultimo'])).time(timestamp)
    
    write_api.write(bucket=bucket, org=organization, record=registro)

# Creamos un hilo para cada consumidor, o en otras palabras, asignamos la funcion ver_mensajes a cada consumidor
threads = []
consumer_group = [consumidor_1, consumidor_2]
for consumidor in consumer_group:
   hilo = threading.Thread(target=ver_mensajes, args=(consumidor,))
   hilo.start()
   threads.append(hilo)

for hilo in threads:
   hilo.join()


{'type': 'record', 'name': 'CryptoCurrency', 'fields': [{'name': 'nombre', 'type': 'string'}, {'name': 'bid', 'type': 'double'}, {'name': 'bid_size', 'type': 'double'}, {'name': 'ask', 'type': 'double'}, {'name': 'ask_size', 'type': 'double'}, {'name': 'daily_change', 'type': 'double'}, {'name': 'daily_change_percentage', 'type': 'double'}, {'name': 'precio_ultimo', 'type': 'double'}, {'name': 'volume', 'type': 'double'}, {'name': 'precio_maximo', 'type': 'double'}, {'name': 'precio_minimo', 'type': 'double'}], '__fastavro_parsed': True, '__named_schemas': {'CryptoCurrency': {'type': 'record', 'name': 'CryptoCurrency', 'fields': [{'name': 'nombre', 'type': 'string'}, {'name': 'bid', 'type': 'double'}, {'name': 'bid_size', 'type': 'double'}, {'name': 'ask', 'type': 'double'}, {'name': 'ask_size', 'type': 'double'}, {'name': 'daily_change', 'type': 'double'}, {'name': 'daily_change_percentage', 'type': 'double'}, {'name': 'precio_ultimo', 'type': 'double'}, {'name': 'volume', 'type': 'do

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:714)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:430)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:430)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio